# This code trains the the model

In [2]:
import torch
import os
import numpy as np
from utils.datasets import get_hash

path_to_data = "../data/dataset_transformed/drone/"

## Create cache file

In [3]:
# Load the data
cache_name = "train"

with open(path_to_data + cache_name + ".txt") as t:
    images = t.readlines()
images = [x.strip("\n") for x in images]

cache_dic = {}
for img in images:
    label_list = np.array([], ndmin=2)
    data = list()
    with open(path_to_data + "labels/" + img[:-3] + "txt") as f:
        for line in f:
            label_list = np.hstack((label_list, np.array([float(x) for x in line.split(" ")], ndmin=2, dtype=np.float32)))

    data.append(label_list)
    data.append(path_to_data + "mask/" + img[2:-3] + "png")
    data.append((640, 480))
    cache_dic[path_to_data + "images/" + img] = data


cache_dic["hash"] = get_hash([path_to_data + "images/" + x for x in images])
cache_dic["results"] = (len(images), 0, 0, 0, 0, len(images), 0, 0, len(images))
cache_dic["version"] = 0.1

torch.save(cache_dic, path_to_data + cache_name + ".cache")



In [4]:
# print my cache
cache_dic = torch.load("../data/dataset_transformed/drone/train.cache")
print(cache_dic)

{'../data/dataset_transformed/drone/images/000046.jpg': [array([[          0,     0.20464,     0.77645,     0.15669,     0.75354,     0.20128,      0.7464,      0.2066,     0.72006,     0.24623,     0.71342,     0.16856,     0.84804,     0.21325,     0.84213,     0.21697,     0.80303,     0.25667,     0.79735,     0.09998,     0.13461,         320,         320,
                640,         480,         320,         240,         640,         480]]), '../data/dataset_transformed/drone/mask/0046.png', (640, 480)], '../data/dataset_transformed/drone/images/000097.jpg': [array([[          0,     0.20574,     0.15233,     0.21741,     0.17168,     0.16705,     0.14723,     0.21703,     0.21436,      0.1716,      0.1939,     0.23027,     0.10214,     0.18187,    0.075602,     0.22872,     0.15017,     0.18489,     0.12791,    0.063215,     0.13876,         320,         320,
                640,         480,         320,         240,         640,         480]]), '../data/dataset_transformed/dr

In [5]:
import torch

cache = torch.load(r"../data/LINEMOD/cat/train.cache")
print(cache)


FileNotFoundError: [Errno 2] No such file or directory: '../data/LINEMOD/cat/train.cache'

## transform dataset 



In [71]:
# Be sure to convert the data to our format focusing on the camera intrinsics
!python data_curation/convert_drone_dataset.py --path_to_dataset ../data/dataset_transformed

## Train

In [ ]:
# By default, the training script will download the COCO pretrained YOLOv5-v7 model. If this is no longer supported, you can download the weights from the YOLOv5-6D google drive (https://drive.google.com/drive/folders/11BW41xO3R1UBnc2Dx1xA3CPbYPGTrfHQ?usp=drive_link) and place them in the weights folder
!python train.py --batch 2 --epochs 1 --cfg models/yolov5s_6dpose_bifpn.yaml --hyp configs/hyp.single.yaml --weights ../data/weights/yolov5s.pt --data configs/linemod/cat.yaml --rect --optimizer Adam 

In [1]:
# By default, the training script will download the COCO pretrained YOLOv5-v7 model. If this is no longer supported, you can download the weights from the YOLOv5-6D google drive (https://drive.google.com/drive/folders/11BW41xO3R1UBnc2Dx1xA3CPbYPGTrfHQ?usp=drive_link) and place them in the weights folder
!python train.py --batch 16 --epochs 100 --cfg models/yolov5s_6dpose_bifpn.yaml --hyp configs/hyp.single.yaml --weights ../data/weights/yolov5s.pt --data configs/drone_data/drone.yaml --rect --optimizer Adam 

YOLOv5 1c813f5 torch 1.9.1+cu102 CPU

Namespace(weights='../data/weights/yolov5s.pt', cfg='models/yolov5s_6dpose_bifpn.yaml', data='configs/drone_data/drone.yaml', hyp='configs/hyp.single.yaml', epochs=100, batch_size=16, img_size=[640, 640], rect=True, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='', single_cls=False, optimizer='Adam', sync_bn=False, local_rank=-1, log_imgs=8, log_artifacts=False, workers=8, project='runs/train', entity=None, name='exp', exist_ok=False, quad=False, linear_lr=False, standard_lr=False, symetric=False, world_size=1, global_rank=-1, save_dir='runs/train/exp3', total_batch_size=16)
Start Tensorboard with "tensorboard --logdir runs/train", view at http://localhost:6006/
hyperparameters: lr0=0.0001, lrf=0.01, lr_factor=0.5, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=1.5, obj=0.1, obj_pw=1.0, cls=0.5, cls_pw=1.0, 